### **Initial Set Up**

Install Necessary Packages

In [2]:
import numpy as np
import pandas as pd
import pymysql as msql
import config
import sql_config
import json
import requests
import warnings
import getpass as gp
import mysql.connector as mysql
import datetime
from datetime import datetime
from mysql.connector import Error
import smtplib, ssl
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import boto3
warnings.filterwarnings("ignore", category=DeprecationWarning)

Set up config.py file with subscriber keys necessary from APIs to access the information.

In [3]:
# Rapid API Subscriber Keys
airquality_key = config.a_key
covid_key = config.c_key
weather_key = config.w_key

# S3 Data Store
AWS_KEY = config.AWS_KEY
AWS_SECRET = config.AWS_SECRET


Password for SQL Connection/Email Connection

In [4]:
#passwd=gp.getpass('Enter Password:')
passwd = sql_config.s_key
epasswd = sql_config.e_key

Input your City of Choice

In [5]:
city = "San Diego"
# city = input("What city are you in?")
city

'San Diego'

#### Air Quality API

In [6]:
# URL Website of Air Quality API
url = "https://air-quality-by-api-ninjas.p.rapidapi.com/v1/airquality"

# Parameter Input
querystring = {"city": city}

# Pulling in of Host and Subscriber Key
headers = {
	"X-RapidAPI-Key": airquality_key,
	"X-RapidAPI-Host": "air-quality-by-api-ninjas.p.rapidapi.com"
}

# Pull Data using request library
a_response = requests.request("GET", url, headers=headers, params=querystring)

print(a_response.text)

{"CO": {"concentration": 270.37, "aqi": 3}, "NO2": {"concentration": 14.05, "aqi": 17}, "O3": {"concentration": 72.96, "aqi": 93}, "SO2": {"concentration": 0.98, "aqi": 1}, "PM2.5": {"concentration": 3.06, "aqi": 9}, "PM10": {"concentration": 5.26, "aqi": 4}, "overall_aqi": 93}


In [7]:
# Convert the response to a dictionary
a_data = a_response.json()

# Create a Pandas dataframe from the dictionary
a_df = pd.DataFrame(a_data)
#a_df

In [8]:
# Remove last row as it is not needed.

a_df = a_df.drop(a_df.index[-1])
a_df

,CO,NO2,O3,SO2,PM2.5,PM10,overall_aqi
concentration,270.37,14.05,72.96,0.98,3.06,5.26,93


In [9]:
# Renaming of Columns since decimals in column names do not work with SQL
a_df = a_df.rename(columns={"PM2.5":"FineParticulateMatter", "PM10":"InhalableParticulateMatter", "overall_aqi":"Overall_AQI"})

# save variable city value to add to dataframe column
new_city_col = [city]

# add new column to the dataframe with the saved value
a_df['city'] = new_city_col


In [10]:
# List of Columns in Air Quality and Data Types
print("Variables in Air Quality are: \n", list(a_df.columns))
print("Data Types are: \n",a_df.dtypes)

Variables in Air Quality are: 
 ['CO', 'NO2', 'O3', 'SO2', 'FineParticulateMatter', 'InhalableParticulateMatter', 'Overall_AQI', 'city']
Data Types are: 
 CO                            float64
NO2                           float64
O3                            float64
SO2                           float64
FineParticulateMatter         float64
InhalableParticulateMatter    float64
Overall_AQI                     int64
city                           object
dtype: object


In [11]:
# Convert Dataframe to Dictionary
a_dict = a_df.to_dict('records')
a_dict

# Convert Dictionary to Tuple Values to feed into SQL
a_tuple = [tuple(d.values()) for d in a_dict]
a_tuple

[(270.37, 14.05, 72.96, 0.98, 3.06, 5.26, 93, 'San Diego')]

In [12]:
# Export to CSV File 

a_df.to_csv('Data/a_csv.csv')


### Covid-19 API

In [13]:
# make the API call
# link to documentation: https://rapidapi.com/axisbits-axisbits-default/api/covid-19-statistics/
url = "https://covid-19-statistics.p.rapidapi.com/reports"

# optional parameter selected to querying the API
querystring = {"city_name":city}

# API credentials
headers = {
	"X-RapidAPI-Key": covid_key,
	"X-RapidAPI-Host": "covid-19-statistics.p.rapidapi.com"
}

# get raw response query from the API
c_response = requests.request("GET", url, headers=headers, params=querystring)
print(c_response.text)

{"data":[{"date":"2023-02-26","confirmed":12084297,"deaths":100816,"recovered":0,"confirmed_diff":0,"deaths_diff":0,"recovered_diff":0,"last_update":"2023-02-27 04:20:54","active":11983481,"active_diff":0,"fatality_rate":0.0083,"region":{"iso":"USA","name":"US","province":"California","lat":"36.1162","long":"-119.6816","cities":[{"name":"San Diego","date":"2023-02-26","fips":6073,"lat":"33.03484597","long":"-116.7365326","confirmed":1064093,"deaths":5768,"confirmed_diff":0,"deaths_diff":0,"last_update":"2023-02-27 04:20:54"}]}}]}


In [14]:
# Convert the response to a dictionary
c_data = c_response.json()

# Create a Pandas dataframe from the dictionary
c_df = pd.DataFrame(c_data)

# Unpack the dictionaries in the column into separate columns
df_unpacked = pd.json_normalize(c_df['data'])

# merge the new columns back into the original dataframe
df_1 = c_df.merge(df_unpacked, left_index=True, right_index=True)

# drop the original column containing the dictionaries
df_2 = df_1.drop('data', axis=1)

# extract the dictionary from region.cities column for flattening
col_dic = df_2['region.cities'].iloc[0]

# turn the extracted dictionary into its on dataframe 
flatten_col = pd.DataFrame(col_dic)

# merge the two dataframes
result = df_2.merge(flatten_col, left_index=True, right_index=True)

# drop columns 
result.drop(['last_update_x', 'active', 'active_diff', 'region.name', 'region.cities', 'date_y'], axis=1, inplace=True)

# rename columns to be more understandable
c_df = result.rename(columns={"date_x": "date", "confirmed_x": "total_confirmed_cases_state","deaths_x": "total_deaths_state", 
                                         "confirmed_diff_x": "confirmed_today_state", "deaths_diff_x": "deaths_today_state",
                                        "recovered_diff": "recovered_today_state", "fatality_rate":"fatality_rate_state","region.iso":"country",
                                        "region.province": "state", "region.lat":"state_latitude", "region.long":"state_longitude", "name": "city",
                                        "fips":"fips_city_code", "lat": "city_latitude", "long": "city_longitude", "confirmed_y":"total_confirmed_cases_city",
                                        "deaths_y": "total_deaths_city", "confirmed_diff_y":"confirmed_today_city", "deaths_diff_y": "deaths_today_city",
                                        "last_update_y": "last_API_update"})

c_df

,date,total_confirmed_cases_state,total_deaths_state,recovered,confirmed_today_state,deaths_today_state,recovered_today_state,fatality_rate_state,country,state,...,state_longitude,city,fips_city_code,city_latitude,city_longitude,total_confirmed_cases_city,total_deaths_city,confirmed_today_city,deaths_today_city,last_API_update
0,2023-02-26,12084297,100816,0,0,0,0,0.0083,USA,California,...,-119.6816,San Diego,6073,33.03484597,-116.7365326,1064093,5768,0,0,2023-02-27 04:20:54


In [15]:
# show the values of all the columns
c_dict = c_df.to_dict('records')
#c_dict

c_tuple = [tuple(d.values()) for d in c_dict]
c_tuple

[('2023-02-26',
  12084297,
  100816,
  0,
  0,
  0,
  0,
  0.0083,
  'USA',
  'California',
  '36.1162',
  '-119.6816',
  'San Diego',
  6073,
  '33.03484597',
  '-116.7365326',
  1064093,
  5768,
  0,
  0,
  '2023-02-27 04:20:54')]

In [16]:
# List of Columns in Covid-19 and Data Types
print("Variables in Covid-19 are: \n", list(c_df.columns))
print("Data Types are: \n",c_df.dtypes)

Variables in Covid-19 are: 
 ['date', 'total_confirmed_cases_state', 'total_deaths_state', 'recovered', 'confirmed_today_state', 'deaths_today_state', 'recovered_today_state', 'fatality_rate_state', 'country', 'state', 'state_latitude', 'state_longitude', 'city', 'fips_city_code', 'city_latitude', 'city_longitude', 'total_confirmed_cases_city', 'total_deaths_city', 'confirmed_today_city', 'deaths_today_city', 'last_API_update']
Data Types are: 
 date                            object
total_confirmed_cases_state      int64
total_deaths_state               int64
recovered                        int64
confirmed_today_state            int64
deaths_today_state               int64
recovered_today_state            int64
fatality_rate_state            float64
country                         object
state                           object
state_latitude                  object
state_longitude                 object
city                            object
fips_city_code                   int64
city

In [17]:
c_csv = c_df.to_csv('Data/c_csv.csv')

### Weather API

In [18]:
# URL Website of Weather API
url = "https://weather-by-api-ninjas.p.rapidapi.com/v1/weather"

# Pulling of Host and Subscriber Key
headers = {
	"X-RapidAPI-Key": weather_key,
	"X-RapidAPI-Host": "weather-by-api-ninjas.p.rapidapi.com"
}

# Parameter Input
querystring = {"city": city}

# Pull Data using request library
w_response = requests.request("GET", url, headers=headers, params = querystring)

print(w_response.text)

{"cloud_pct": 75, "temp": 12, "feels_like": 11, "humidity": 80, "min_temp": 10, "max_temp": 13, "wind_speed": 3.6, "wind_degrees": 210, "sunrise": 1677507548, "sunset": 1677548637}


In [19]:
# Convert the response to a dictionary
w_data = w_response.json()

# Create a Pandas dataframe from the dictionary
w_df = pd.DataFrame(w_data, index = [0])

In [20]:
# Convert Unix Timestamp to Readable Datetime
w_df['sunrise'] = datetime.fromtimestamp(w_df['sunrise']).strftime('%Y-%m-%d %H:%M:%S')
w_df['sunset'] = datetime.fromtimestamp(w_df['sunset']).strftime('%Y-%m-%d %H:%M:%S')


In [21]:
# Convert Temperatures from Celsius to Fahrenheit
w_df['temperature'] = w_df.apply(lambda x: (9/5)*x['temp']+32,axis=1)
w_df['temp_feels_like'] = w_df.apply(lambda x: (9/5)*x['feels_like']+32,axis=1)
w_df['min_temperature'] = w_df.apply(lambda x: (9/5)*x['min_temp']+32,axis=1)
w_df['max_temperature'] = w_df.apply(lambda x: (9/5)*x['max_temp']+32,axis=1)

In [22]:
# save variable city value to add to dataframe column
new_city_col = [city]

# add new column to the dataframe with the saved value
w_df['city'] = new_city_col
w_df

,cloud_pct,temp,feels_like,humidity,min_temp,max_temp,wind_speed,wind_degrees,sunrise,sunset,temperature,temp_feels_like,min_temperature,max_temperature,city
0,75,12,11,80,10,13,3.6,210,2023-02-27 06:19:08,2023-02-27 17:43:57,53.6,51.8,50.0,55.4,San Diego


In [23]:
# Preview Dataframe
# Change order of the columns for the preview, based on the average user needs
w_df = w_df.iloc[:,[10, 11, 12, 13, 0, 3, 6, 7, 8, 9, 14]]
w_df

,temperature,temp_feels_like,min_temperature,max_temperature,cloud_pct,humidity,wind_speed,wind_degrees,sunrise,sunset,city
0,53.6,51.8,50.0,55.4,75,80,3.6,210,2023-02-27 06:19:08,2023-02-27 17:43:57,San Diego


In [24]:
# List of Columns in Weather and Data Types
print("Variables in Weather are: \n", list(w_df.columns))
print("Data Types are: \n",w_df.dtypes)

Variables in Weather are: 
 ['temperature', 'temp_feels_like', 'min_temperature', 'max_temperature', 'cloud_pct', 'humidity', 'wind_speed', 'wind_degrees', 'sunrise', 'sunset', 'city']
Data Types are: 
 temperature        float64
temp_feels_like    float64
min_temperature    float64
max_temperature    float64
cloud_pct            int64
humidity             int64
wind_speed         float64
wind_degrees         int64
sunrise             object
sunset              object
city                object
dtype: object


In [25]:
# Convert to Dictionary
w_dict = w_df.to_dict('records')
#w_dict

# Convert to Tuple for feeding into SQL Table
w_tuple = [tuple(d.values()) for d in w_dict]
w_tuple

[(53.6,
  51.8,
  50.0,
  55.400000000000006,
  75,
  80,
  3.6,
  210,
  '2023-02-27 06:19:08',
  '2023-02-27 17:43:57',
  'San Diego')]

In [26]:
w_csv = w_df.to_csv('Data/w_csv.csv')

Save All Output CSV Files for Time Stamp Logs

In [27]:
import datetime
a_df.to_csv(f'Data/PythonCSVTimeLogs/a_csv{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv')
w_df.to_csv(f'Data/PythonCSVTimeLogs/w_csv{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv')
c_df.to_csv(f'Data/PythonCSVTimeLogs/c_csv{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv')


### **Load Python to SQL Database**

#### Create Database

In [28]:
# Create New Database
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd)
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("CREATE DATABASE public_safety")
        print("Database is created")
except Error as e:
    print("Error while connecting to MySQL", e)

Error while connecting to MySQL 1007 (HY000): Can't create database 'public_safety'; database exists


#### Add Air Quality Table to Public Safety Database

In [29]:
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd, database= 'public_safety')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS air_quality;')
        print('Creating Table Air Quality')
# in the below line please pass the create table statement which you want #to create
        cursor.execute('CREATE TABLE air_quality(CO float, NO2 float, O3 float, SO2 float, FineParticulateMatter float, InhalableParticulateMatter float, Overall_AQI SMALLINT, city text)')
        conn.commit()
        print("Table is created....")
        #loop through the data frame
            #here %S means string values 
        a_sql = 'INSERT INTO air_quality (CO, NO2, O3, SO2, FineParticulateMatter, InhalableParticulateMatter, Overall_AQI, city) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)'
        cursor.executemany(a_sql, a_tuple)
        print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
        print("Table is Complete")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('public_safety',)
Creating Table Air Quality
Table is created....
Record inserted
Table is Complete


#### Transform Data in SQL for Air Quality

In [30]:
a_query= pd.read_sql(""" 
SELECT CO,
    NO2,
    O3,
    SO2,
    FineParticulateMatter,
    InhalableParticulateMatter,
    Overall_AQI,
    city AS City
FROM air_quality;
 """, conn)
 
a_query

c:\Users\trevor.sauerbrey\WPy64-3890\python-3.8.9.amd64\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,CO,NO2,O3,SO2,FineParticulateMatter,InhalableParticulateMatter,Overall_AQI,City
0,270.37,14.05,72.96,0.98,3.06,5.26,93,San Diego


In [31]:
# Export Query as CSV File

sql_adf = pd.DataFrame(a_query)

a_sql_csv = sql_adf.to_csv('Data/a_sql_csv.csv')

#### Add Weather Table to Public Safety Database

In [32]:
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd, database= 'public_safety')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS weather;')
        print('Creating Table Weather')
# in the below line please pass the create table statement which you want #to create
        cursor.execute('CREATE TABLE weather(temperature float, temp_feels_like float, min_temperature float, max_temperature float, cloud_pct SMALLINT, humidity SMALLINT, wind_speed float, wind_degrees SMALLINT, sunrise datetime, sunset datetime, city text)')
        conn.commit()
        print("Table is created....")
        #loop through the data frame
            #here %S means string values 
        w_sql = 'INSERT INTO weather (temperature, temp_feels_like, min_temperature, max_temperature, cloud_pct, humidity, wind_speed, wind_degrees, sunrise, sunset, city) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        cursor.executemany(w_sql, w_tuple)
        print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
        print("Table is Complete")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('public_safety',)
Creating Table Weather
Table is created....
Record inserted
Table is Complete


#### Transform Data in SQL for Weather

In [33]:
w_query= pd.read_sql(""" 
SELECT * FROM weather;
 """, conn)
 
w_query

c:\Users\trevor.sauerbrey\WPy64-3890\python-3.8.9.amd64\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,temperature,temp_feels_like,min_temperature,max_temperature,cloud_pct,humidity,wind_speed,wind_degrees,sunrise,sunset,city
0,53.6,51.8,50.0,55.4,75,80,3.6,210,2023-02-27 06:19:08,2023-02-27 17:43:57,San Diego


In [34]:
# Export Query as CSV File

sql_wdf = pd.DataFrame(w_query)

w_sql_csv = sql_wdf.to_csv('Data/w_sql_csv.csv')

#### Add Covid-19 Table to Public Safety Database

In [35]:
try:
    conn=mysql.connect(host='localhost',port=int(3306),user='root',passwd=passwd, database= 'public_safety')
    if conn.is_connected():
        cursor = conn.cursor()
        cursor.execute("select database();")
        record = cursor.fetchone()
        print("You're connected to database: ", record)
        cursor.execute('DROP TABLE IF EXISTS covid;')
        print('Creating Table Covid-19')
# in the below line please pass the create table statement which you want to create
        cursor.execute('CREATE TABLE covid(date date, total_confirmed_cases_state INT, total_deaths_state INT, recovered INT, confirmed_today_state INT, deaths_today_state INT, recovered_today_state INT, fatality_rate_state float, country text, state text, state_latitude float, state_longitude float, city text, fips_city_code INT, city_latitude float, city_longitude float, total_confirmed_cases_city INT, total_deaths_city INT, confirmed_today_city INT, deaths_today_city INT, last_API_update datetime)')
        conn.commit()
        print("Table is created....")
        #loop through the data frame
            #here %S means string values 
        c_sql = 'INSERT INTO covid (date, total_confirmed_cases_state, total_deaths_state, recovered, confirmed_today_state, deaths_today_state, recovered_today_state, fatality_rate_state, country, state, state_latitude, state_longitude, city, fips_city_code, city_latitude, city_longitude, total_confirmed_cases_city, total_deaths_city, confirmed_today_city, deaths_today_city, last_API_update) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)'
        cursor.executemany(c_sql, c_tuple)
        print("Record inserted")
            # the connection is not auto committed by default, so we must commit to save our changes
        conn.commit()
        print("Table is Complete")
except Error as e:
            print("Error while connecting to MySQL", e)

You're connected to database:  ('public_safety',)
Creating Table Covid-19
Table is created....
Record inserted
Table is Complete


#### Transform Data in SQL for Covid-19
Transform Covid-19 Table to select only data specifically related to the city to be cohesive with the air quality and weather APIs that only show city data.

In [36]:
# Transform to query only city related information and information users would need
c_query= pd.read_sql(""" 
SELECT date as Date,
    total_confirmed_cases_city AS Total_Confirmed_Cases,
    total_deaths_city AS Total_Covid_Related_Deaths,
    confirmed_today_city AS Confirmed_Positive_Cases_Today,
    deaths_today_city AS Covid_Related_Deaths_Today,
    fips_city_code AS FIPS_City_Code,
    city AS City
FROM covid;
 """, conn)
 
c_query

c:\Users\trevor.sauerbrey\WPy64-3890\python-3.8.9.amd64\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,Date,Total_Confirmed_Cases,Total_Covid_Related_Deaths,Confirmed_Positive_Cases_Today,Covid_Related_Deaths_Today,FIPS_City_Code,City
0,2023-02-26,1064093,5768,0,0,6073,San Diego


In [37]:
# Export Query as CSV File

sql_cdf = pd.DataFrame(c_query)

c_sql_csv = sql_cdf.to_csv('Data/c_sql_csv.csv')

#### Load CSV Files to S3 Data Storage

In [38]:
# S3 Bucket Name
bucket = "ads507publicsafety"

# Make Connection to S3 for Writing in Data to it
s3_open = boto3.client('s3',
                        aws_access_key_id = AWS_KEY,
                        aws_secret_access_key = AWS_SECRET)
a_filename = 'Data/a_sql_csv.csv'
w_filename = 'Data/w_sql_csv.csv'
c_filename = 'Data/c_sql_csv.csv'
a_csv_buffer = sql_adf.to_csv(index=False)
w_csv_buffer = sql_wdf.to_csv(index=False)
c_csv_buffer = sql_cdf.to_csv(index=False)

# Push CSV Files to S3 Bucket
s3_open.put_object(Bucket = bucket, Key = a_filename, Body = a_csv_buffer)
s3_open.put_object(Bucket = bucket, Key = w_filename, Body = w_csv_buffer)
s3_open.put_object(Bucket = bucket, Key = c_filename, Body = c_csv_buffer)

{'ResponseMetadata': {'RequestId': '2S6QJC0C85EMJP57',
  'HostId': 'iG8cU6tJ/4djEK5s2CARdMOFzeYTpCvxMYQ+Hyh/uSeoBhZMNSBm/64+T4FnucreeC/2oSRmtuM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'iG8cU6tJ/4djEK5s2CARdMOFzeYTpCvxMYQ+Hyh/uSeoBhZMNSBm/64+T4FnucreeC/2oSRmtuM=',
   'x-amz-request-id': '2S6QJC0C85EMJP57',
   'date': 'Tue, 28 Feb 2023 04:22:44 GMT',
   'x-amz-version-id': 'n5nXkZPpT9kfLQoZ_aU_e1LzYi2MitzM',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"f248dc36763e36d01a197dd51e71b45a"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 1},
 'ETag': '"f248dc36763e36d01a197dd51e71b45a"',
 'ServerSideEncryption': 'AES256',
 'VersionId': 'n5nXkZPpT9kfLQoZ_aU_e1LzYi2MitzM'}

Export CSV Files from SQL Queries to Record Keeping Time Logs

In [39]:
sql_adf.to_csv(f'Data/SQLCSVTimeLogs/a_csv{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv')
sql_wdf.to_csv(f'Data/SQLCSVTimeLogs/w_csv{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv')
sql_cdf.to_csv(f'Data/SQLCSVTimeLogs/c_csv{datetime.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")}.csv')

#### Automated E-Mail Feature

For any complications with e-mail errors. Please refer to these links. It could be that your e-mail is a school or work email, or does not have 2-Step Authentication turned on. An APP Password is needed if you receive an error.

https://support.google.com/accounts/answer/185833?hl=en

https://joequery.me/guides/python-smtp-authenticationerror/

In [40]:
# Email Password for Sender Email
emailpw=epasswd

In [41]:
# Set up Connection to Email Parameters
smtp_server = "smtp.gmail.com"
port = 587  # For starttls
sender_email = "khu@sandiego.edu"
recipients = ["khu@sandiego.edu", "gtashchyan@sandiego.edu", "tsauerbrey@sandiego.edu"]
password = emailpw

message = MIMEMultipart("alternative")
message["Subject"] = "Public Safety Daily Updates"
message["From"] = sender_email
message["To"] = ", ".join(recipients)


In [42]:
# Create the plain-text and HTML version of your message
text = """\
Hello,

Here are your daily public safety updates.
"""
html = """\
<html>
  <body>
    <p style="color:Black;">Hello,<br><br>
       Here are your daily public safety updates.
  </body>
</html>
"""

# Format Output of Air Quality SQL Query to HTML 
a_html = """\
<html>
  <head></head>
  <body>
  <p style="color:Black;"><b>Air Quality and Pollutant Concentration</b></p>
    {0}
  <br>
      <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSqEWsXN5DNGa97qL9jXCDqoi5KutgfzzWM-w&usqp=CAU" alt="img" />
  <br/>
  </body>
</html>
""".format(sql_adf.to_html())

# Format Output of Weather SQL Query to HTML 
w_html = """\
<html>
  <head></head>
  <body>
  <p style="color:Black;"><b>Weather</b></p>
    {0}
  <br>
  </body>
</html>
""".format(sql_wdf.to_html())

# Format Output of Covid-19 SQL Query to HTML 
c_html = """\
<html>
  <head></head>
  <body>
  <p style="color:Black;"><b>Covid-19 Statistics</b></p>
    {0}
  </body>
  <br>
</html>
""".format(sql_cdf.to_html())

# Add Footer for auto-generated email.
footer_html = """\
<html>
  <body>
  <br><br>
    <p style="color:red;"><b>Please do not reply to this email as it is auto-generated. </b></p>
    </p>
  </body>
</html>
"""

# Turn these into plain/html MIMEText objects
part1 = MIMEText(text, "plain")
part2 = MIMEText(html+a_html+w_html+c_html+footer_html, "html")

# Add HTML/plain-text parts to MIMEMultipart message
# The email client will try to render the last part first
message.attach(part1)
message.attach(part2)

# Create secure connection with server and send email
context = ssl.create_default_context()
with smtplib.SMTP_SSL("smtp.gmail.com", 465, context=context) as server:
    server.login(sender_email, emailpw)
    server.sendmail(
        sender_email, recipients, message.as_string()
    )
print("Email has been sent!")

Email has been sent!
